In [ ]:
!python -m spacy download en_core_web_lg

2023-04-17 00:10:28.952555: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import os
import csv
import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import statistics
from statistics import mode

nlp = spacy.load("en_core_web_lg")

In [ ]:
path = "/content/data"
dir_list = os.listdir(path)
os.chdir("/content/data")

universal_new_lines = '\n' + '\r' + '\r\n' + '\v' + '\x0b' + '\f' + '\x0c' + '\x1c' + '\x1d' + '\x1e' + '\x85' + '\u2028' + '\u2029' 
pun = string.punctuation + '\t' + '—' + '‘’' + universal_new_lines
label_list = ['PERSON', 'ORG', 'GPE']
fields = ['FILE', 'PERSON', 'ORG', 'GPE', 'YEAR', 'AGENCY']
processed_data = []

In [ ]:
def data_processing(data):
  # removing punctuations and new lines and tabs
  for sym in pun:
    cnt = data.count(sym)
    data = data.replace(sym, '', cnt)

  # lower the data
  data = data.lower()
  doc = nlp(data)

  token_classification = {'FILE':'NA','PERSON':'NA', 'ORG':'NA', 'GPE':'NA', 'YEAR':'NA', 'AGENCY':'NA'}

  year = []
  for token in doc:
    if token.pos_ == 'NUM' and token.shape_ == 'dddd':
      year.append(token.text)

  if len(year) == 0:
    token_classification['YEAR'] = 'NA'
  else:
    token_classification['YEAR'] = mode(year)

  # fetching entities and keywords
  person = []
  org = []
  gpe = []
  for ent in doc.ents:
    if ent.label_ in label_list:
      if ent.label_ == 'PERSON':
        person.append(ent.text)
      elif ent.label_ == 'ORG':
        org.append(ent.text)
      elif ent.label_ == 'GPE':
        gpe.append(ent.text)

  if len(person) == 0:
    token_classification['PERSON'] = 'NA'
  else:
    token_classification['PERSON'] = mode(person)

  if len(org) == 0:
    token_classification['ORG'] = 'NA'
  else:
    token_classification['ORG'] = mode(org)

  if len(gpe) == 0:
    token_classification['GPE'] = 'NA'
  else:
    token_classification['GPE'] = mode(gpe)

  return token_classification

In [ ]:
for fname in dir_list:

  file = open(fname)
  c = 0
  content_list = file.readlines()
  while '\n' in content_list:
    content_list.remove('\n')
  for content in content_list:
    c = c + 1
    token_classification = data_processing(content)
    token_classification['FILE'] = fname.replace("_docs.txt", str(c))
    token_classification['AGENCY'] = fname.replace("_docs.txt", '')
    processed_data.append(token_classification)
# print(processed_data)

with open("/content/ProcessedData.csv", 'w') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = fields)
  writer.writeheader()
  writer.writerows(processed_data)